**Title**: Delete Empty Container with Search <br>
**Date**:  09/08/2020 <br>
**Description**:  
* This notebook will demonstrates how to delete empty acquisitions, sessions and subjects container if only they are empty. 
* Delete containers by searching for any acquisitions that have no files and remove the parent sessions and subjects container as well. 

# Install and Import Dependencies

In [ ]:
# Install specific packages required for this notebook
!pip install 'flywheel-sdk>=12.0.0' 

In [1]:
# Import packages
from getpass import getpass
import logging
import datetime
from pathlib import Path
import os
import csv
from tqdm.notebook import tqdm

import flywheel
from permission import check_user_permission
import pandas as pd

In [2]:
 # Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

# Flywheel API Key and Client


Get your API_KEY. More on this at in the Flywheel SDK doc [here](https://flywheel-io.gitlab.io/product/backend/sdk/branches/master/python/getting_started.html#api-key).

In [ ]:
API_KEY = getpass('Enter API_KEY here: ')

Instantiate the Flywheel API client

In [ ]:
fw = flywheel.Client(API_KEY if 'API_KEY' in locals() else os.environ.get('FW_KEY'))

del API_KEY

Show Flywheel logging information

In [ ]:
log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

# Overview

- How this notebook differs from the top-down delete containers version?
- Able to iterate through each acquisition to grab a list of empty acquisition -> then delete the acq and its parent session and subject container.

# Requirements

In order to run this notebook, you will need to have the right permission on the Group level to create a new Project for testing.

In [3]:
# Minimum requirements that you will need to remove containers on the Project level. 
min_reqs = {
"site": "user",
"group": "admin"
}

In [ ]:
GROUP_ID = input('Please enter the Group ID that you will be using to create the new project: ')

In [ ]:
check_user_permission(fw, min_reqs, group = GROUP_ID)

# Initialize a few values
Now, we will be defining a few values that will be use in this notebook. The `GROUP_ID` is the Group ID that you will be using throughout this notebook. 


In [ ]:
GROUP_ID = input('Please enter the Group that you have admin permission for')

In [ ]:
PROJECT_LABEL = 'test-delete-containers-project'

In [ ]:
path_to_test_file = Path("source_folder/to_your/test_file.dicom.zip")

In [4]:
dummy_filename = '/tmp/dummy.txt'
filename = 'dummy.txt'

In [ ]:
with open(dummy_filename, 'w') as fp:
    fp.writelines(['Just a dummy text file.'])

# Create a Dummy Project

In [ ]:
my_group = fw.lookup(GROUP_ID)

In [ ]:
project=my_group.add_project(label=PROJECT_LABEL)

- Talk about having a function `add_subject` to add subject to the dummy project


In [ ]:
def add_subject(subj_label, session_label, acq_label):
    """Add a subject with empty session and acquisition container.
    
    Args:
    
    Return:
    
    """
    subject = project.add_subject(label=subj_label)
    session = subject.add_session(label=session_label)
    acquisition = session.add_acquisition(label=acq_label)
    return subject

First, we will be adding new subjects with a session and an acquisition that have no files.

In [ ]:
# Empty dict to store subject container
subjects = {}

for subj in ['hooky', 'mooky', 'zooky', 'kooky', 'oooky', 'looky', 'pooky']:
    subjects[subj] = add_subject(subj, f"{subj}-sess", f"{subj}-acq")
    print(f"added {subj}")

## Set Up Dummy Project

Add a Session with multiple Acquisitions that contain no files to the first Subject `hooky`. 

In [ ]:
session = subjects['hooky'].add_session(label='test-multiple-acq')
acquisition_1 = session.add_acquisition(label='first-acq')
acquisition_2 = session.add_acquisition(label='second-acq')

For the second Subject, `mooky`, we will be adding a Session with multiple Acquisitions which contain some files. 

In [ ]:
session = subjects['mooky'].add_session(label='test-multiple-acq-with-files')
acquisition_1 = session.add_acquisition(label='first-acq')
acquisition_1.upload_file(path_to_test_file)
acquisition_2 = session.add_acquisition(label='second-acq')
acquisition_2.upload_file(dummy_filename) 

As for the third Subject, `zooky`, a Session will be added with an empty Acquisition and a file will be uploaded to the Session container. 

In [ ]:
session = subjects['zooky'].add_session(label='test-session-has-file')
acquisition_1 = session.add_acquisition(label='first-acq')
acquisition_2 = session.add_acquisition(label='second-acq')
session.upload_file(dummy_filename)

The fourth Subject, `kooky`, will have two Session container with two Acquisitions and one file will be uploaded to the Subject container

In [ ]:
subjects['kooky'].upload_file(dummy_filename)
session = subjects['kooky'].add_session(label='test-multiple-sess1')
acquisition_1 = session.add_acquisition(label='first-acq')
acquisition_2 = session.add_acquisition(label='second-acq')

session_2 = subjects['kooky'].add_session(label='kooky-test-multiple-sess2')
acquisition_1 = session_2.add_acquisition(label='kooky-ses2-first-acq')
acquisition_2 = session_2.add_acquisition(label='kooky-ses2-second-acq')